In [14]:
import folium
import pandas as pd

In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AfJohXkls9U_yHXfpY1XoKT7zdAIWjLgsyOBSu06qMBgGea_FrNVxZzx24A



Successfully saved authorization token.


In [3]:
# Load the ImageCollection
gsw_monthly = ee.ImageCollection('JRC/GSW1_4/MonthlyHistory')
print('gsw monthly:', gsw_monthly)

gsw monthly: ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "ImageCollection.load",
    "arguments": {
      "id": {
        "constantValue": "JRC/GSW1_4/MonthlyHistory"
      }
    }
  }
})


In [4]:
# Function to calculate monthly max values within each year
yearlist = ee.List.sequence(2012, 2017)
reducer = ee.Reducer.max()

In [5]:
# Function to create annual composites
def create_yearly_composite(year):
    year = ee.Number(year)
    yearCol = gsw_monthly.filter(ee.Filter.calendarRange(year, year, 'year'))
    yearMax = yearCol.reduce(reducer)

    imgList = yearCol.aggregate_array('system:index')
    n_img = imgList.size()
    nBands = yearMax.bandNames().size()

    return yearMax.set({
        'year': year,
        'image_list': imgList,
        'n_bands': nBands,
        'n_img': n_img,
        'method': 'max value',
        'system:time_start': year
    })

In [6]:
yearCompList = yearlist.map(create_yearly_composite)


In [7]:
# Convert the annual composite image list to an ImageCollection
annual_max_water = ee.ImageCollection.fromImages(yearCompList).filter(ee.Filter.gt('n_bands', 0))
print('annual_max_water', annual_max_water)

# Convert to list
annual_list = annual_max_water.toList(annual_max_water.size())
print('annual_list', annual_list)

annual_max_water ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "ImageCollection.fromImages",
          "arguments": {
            "images": {
              "functionInvocationValue": {
                "functionName": "List.map",
                "arguments": {
                  "baseAlgorithm": {
                    "functionDefinitionValue": {
                      "argumentNames": [
                        "_MAPPING_VAR_0_0"
                      ],
                      "body": {
                        "functionInvocationValue": {
                          "functionName": "Element.set",
                          "arguments": {
                            "key": {
                              "constantValue": "system:time_start"
                            },
                            "object": {
                              "funct

In [8]:
# Function to calculate the difference between two images
def difference(img1, img2):
    def mask_only_water(img):
        return img.updateMask(img.eq(2)).multiply(0).add(1).unmask(0)

    img1_mask = mask_only_water(img1)
    img2_mask = mask_only_water(img2)

    diff = img2_mask.subtract(img1_mask).copyProperties(img1_mask, ['max value', 'n_img', 'system:time_start', 'year'])
    img_diff = ee.Image(diff)
    return img_diff

In [9]:
year1L = 2012
year2L = 2013

imgtest1 = ee.Image(annual_list.get(year1L - 2012))
print(imgtest1)
imgtest2 = ee.Image(annual_list.get(year2L - 2012))

diff = difference(imgtest1, imgtest2)
print(diff)

ee.Image({
  "functionInvocationValue": {
    "functionName": "List.get",
    "arguments": {
      "index": {
        "constantValue": 0
      },
      "list": {
        "functionInvocationValue": {
          "functionName": "Collection.toList",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "ImageCollection.fromImages",
                      "arguments": {
                        "images": {
                          "functionInvocationValue": {
                            "functionName": "List.map",
                            "arguments": {
                              "baseAlgorithm": {
                                "functionDefinitionValue": {
                                  "argumentNames": [
                                 

In [10]:
!pip install folium

In [13]:
# Set visualization parameters.
vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'blue', 'green']
}

# Create a folium map object.
center = [40.7128, -74.0060]  # Example: New York City's latitude and longitude.
m = folium.Map(location=center, zoom_start=6)

# Add the layer to the map.
diff_layer = ee.Image(diff).visualize(**vis_params)
map_id_dict = ee.Image(diff_layer).getMapId()
folium.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='difference',
).add_to(m)

# Add a layer control panel to the map.
folium.LayerControl().add_to(m)

# Display the map.
m

In [21]:
# import ee
# ee.Initialize()

# Define the corners of the large ROI
top_left = (70, 45)
bottom_right = (110, 23)

# Define the grid size
cols = 5  # Number of columns
rows = 4  # Number of rows

# Calculate the step size for each grid cell
width_step = (bottom_right[0] - top_left[0]) / cols
height_step = (top_left[1] - bottom_right[1]) / rows


In [22]:
def create_grid(top_left, bottom_right, width_step, height_step, cols, rows):
    grid = []
    for i in range(cols):
        for j in range(rows):
            # Calculate corners of the cell
            left = top_left[0] + i * width_step
            right = left + width_step
            top = top_left[1] - j * height_step
            bottom = top - height_step

            # Create a rectangle for this cell
            cell = ee.Geometry.Rectangle([left, bottom, right, top])
            grid.append(cell)
    return grid

grid = create_grid(top_left, bottom_right, width_step, height_step, cols, rows)


In [ ]:
for i, section in enumerate(grid):
    task = ee.batch.Export.image.toDrive(**{
        'image': diff,  # Replace with your 'difference' image
        'description': f'High_Mountain_Asia_Difference_Section_{i}',
        'folder': 'EarthEngineData',
        'scale': 30,  # Adjust as necessary
        'region': section,
        'fileFormat': 'GeoTIFF',
        'maxPixels': 1e13
    })
    task.start()
